In [1]:
from crewai import LLM
import os
import json
import re

In [2]:
# Load environment variables
causal_template = json.load(open("input/2_causal_interpretability.json"))

In [3]:
llm = LLM(

    #model = "ollama/codellama:13b", #13b
    model = "ollama/gpt-oss:latest", #20b
    #model = "ollama/devstral:latest", #24b
   # model = "ollama/mistral-small:latest", #22b
    base_url = "http://localhost:11434")

## Scenario A: Code generation text2code task

### Step 1: Define the task

In [4]:
messages_a = causal_template['scenario_a']['step1']['messages']

In [5]:
messages_a

[{'role': 'system',
  'content': 'You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.'},
 {'role': 'user',
  'content': '[CAUSAL TASK] Giving the following variable in the json file {FACTORS}, generate a causal graph that explains how they influence each other. Consider that those variables are influencing the generation of the function {FUNCTION}, therefore the code generation should be a separate node in the graph. The outcome must be in Graphviz DOT format.'},
 {'role': 'user',
  'content': '[COUNTERFACTUAL TASK] What would have changed or happened if the software developer had not considered the variable {NODE} during the generation of the {FUNCTION} function? Provide at most three counterfactual answers. Estimate a probability of occurrence for each answer based solely on the causal graph in Graphviz DOT format. The outcome must be contained in a json format in which 

In [6]:
#messages_a[1]['content'] =' '.join([messages_a[0]['content'], messages_a[1]['content']])

In [7]:
#messages_a[1]['content']

In [6]:
def extract_function_name(response):
    match = re.search(r"def\s+(\w+)\s*\(", response)
    if match:
        return match.group(1)
    return None

In [26]:
def process_messages(message,function, json_factors,node, test):
    #new_message= re.sub(r"\[.*?\]","", message['content'])
    new_message= message['content']
    if function:
        new_message = re.sub(r"\{FUNCTION\}",function, new_message)
    if json_factors:
        new_message = re.sub(r"\{FACTORS\}",json_factors, new_message)
    if re.search(r"\{NODE\}", new_message):
        new_message = re.sub(r"\{NODE\}",node, new_message)
    if re.search(r"\{TEST\}", new_message):
        new_message = re.sub(r"\{TEST\}",test, new_message)
       
    return new_message

In [8]:
def find_name(obj):
    if isinstance(obj, dict):
        for k, v in obj.items():
            if k == "name":
                yield v
            else:
                yield from find_name(v)
    elif isinstance(obj, list):
        for item in obj:
            yield from find_name(item)

In [24]:
def process_introspection_factors(introspection_chat_history):
    factors = []
    function = None
    function_message = introspection_chat_history[3]['content']
    function = extract_function_name(function_message)
    factors_message = introspection_chat_history[5]['content']
    cleaned = re.sub(r"^```json\s*|\s*```$", "", factors_message.strip(), flags=re.MULTILINE)

    data = json.loads(cleaned)
    factors=list(find_name(data))
    return function, function_message, factors, factors_message

In [32]:
import copy
chat_history = []
last_response = ''
messagges = copy.deepcopy(messages_a)
input_file= 'ollama_gpt-oss:latest_step1_step2_sptep3_15.json'
introspection_messaages =json.load(open("output/introspection/"+input_file))['scenario_a']['step1']['messages']

func, func_name, nodes, factors_message = process_introspection_factors(introspection_messaages)


first_node = nodes.pop(0)
last_message = {}
def run(message, node):
    
    message['content'] = process_messages(message, func, factors_message, node)
    print(message['content'])
    print("-----")
    chat_history.append(message)
    last_response = llm.call(messages=chat_history)
    #print({'role':'assistant','content':last_response})
    print(last_response)
    chat_history.append({"role": "assistant", "content": last_response})

for message in messagges:
    last_message = copy.deepcopy(message)
    run(message , first_node)

while nodes:
    print(last_message)
    run(last_message, nodes.pop(0))


TypeError: process_messages() missing 1 required positional argument: 'test'

In [11]:
chat_history


[{'role': 'system',
  'content': 'You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.'},
 {'role': 'assistant',
  'content': 'Sure thing! To get started, could you share the list of variables you’re interested in and the context (e.g., a study design, a real‑world system, or a hypothetical scenario)? Once I know the variables and any known relationships or constraints, I can:\n\n1. Sketch a directed acyclic graph (DAG) that represents the plausible causal structure.\n2. Identify key paths, colliders, and potential confounders.\n3. Draft counterfactual explanations for a few key interventions or events in that system.\n\nFeel free to provide any additional background (e.g., data sources, domain knowledge, or specific questions you want to answer) so I can tailor the DAG and explanations accordingly.'},
 {'role': 'user',
  'content': '[CAUSAL TASK] Giving the following varia

In [12]:
messages_a

[{'role': 'system',
  'content': 'You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.'},
 {'role': 'user',
  'content': '[CAUSAL TASK] Giving the following variable in the json file {FACTORS}, generate a causal graph that explains how they influence each other. Consider that those variables are influencing the generation of the function {FUNCTION}, therefore the code generation should be a separate node in the graph. The outcome must be in Graphviz DOT format.'},
 {'role': 'user',
  'content': '[COUNTERFACTUAL TASK] What would have changed or happened if the software developer had not considered the variable {NODE} during the generation of the {FUNCTION} function? Provide at most three counterfactual answers. Estimate a probability of occurrence for each answer based solely on the causal graph in Graphviz DOT format. The outcome must be contained in a json format in which 

In [14]:
from IPython import get_ipython

# Get the current execution count
exec_count = get_ipython().execution_count
print("This cell run number is:", exec_count)

This cell run number is: 14


In [15]:
output ={
    'scenario_a': {
        'step1': {'messages': chat_history}
    }
}
file = "output/interpretability/"+ llm.model.replace("/","_")+"_step1_"+ str(exec_count) +".json"
with open(file, "w") as f:
    json.dump(output, f, indent=4)  

# Scenario A - interpretability - step 2

In [17]:
messages_2 = causal_template['scenario_a']['step2']['messages']

In [18]:
messages_2

[{'role': 'system',
  'content': 'You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.'},
 {'role': 'user',
  'content': '[CAUSAL TASK] Giving the following variables in the json file {FACTORS}, generate a causal graph that explains how they influence each other. Consider that those variables are influencing the generation of the test case {TEST}, therefore the code generation should be a separate node in the graph. The outcome must be in Graphviz DOT format.'},
 {'role': 'user',
  'content': '[COUNTERFACTUAL TASK] What would have changed or happened if the software tester had not considered the variable {NODE} during the generation of the {TEST} test? Provide at most three counterfactual answers. Estimate a probability of occurrence for each answer based solely on the causal graph in Graphviz DOT format. The outcome must be contained in a json format in which the first lev

In [ ]:
import copy
chat_history_2 = []
last_response = ''
messagges = copy.deepcopy(messages_2)
input_file= 'ollama_gpt-oss:latest_step1_step2_sptep3_15.json'
introspection_messaages =json.load(open("output/introspection/"+input_file))['scenario_a']['step2']['messages']

func_name, func, nodes, factors_message = process_introspection_factors(introspection_messaages)


first_node = nodes.pop(0)
last_message = {}
def run(message, node):
    
    message['content'] = process_messages(message, None, factors_message, node, func)
    print(message['content'])
    print("-----")
    chat_history_2.append(message)
    last_response = llm.call(messages=chat_history_2)
    #print({'role':'assistant','content':last_response})
    print(last_response)
    chat_history_2.append({"role": "assistant", "content": last_response})

for message in messagges:
    last_message = copy.deepcopy(message)
    run(message , first_node)

-while nodes:
    print(last_message)
    run(copy.deepcopy(last_message), nodes.pop(0))


You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.
-----
Got it! To get started on building a causal graph and drafting counterfactual explanations, I’ll need a bit more information from you:

1. **Variables**  
   - List all the variables you’re interested in (e.g., `X`, `Y`, `Z`, etc.).  
   - If possible, give short definitions or notes on what each variable represents (e.g., “`Age` – age of the participant”, “`Treatment` – whether a subject received drug A” …).

2. **Domain Knowledge / Prior Knowledge**  
   - Any known causal relationships (e.g., “Age → Treatment” or “Gender → Outcome”).  
   - Any constraints (e.g., “`Outcome` cannot cause `Treatment` because treatment is administered first”).  
   - Any known confounders or mediators.

3. **Data / Observational Facts**  
   - If you have a dataset, a summary (means, correlations) or any patterns you’ve noticed.  
 

chat_history_2

In [ ]:
chat_history_2

[{'role': 'system',
  'content': 'You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.'},
 {'role': 'assistant',
  'content': 'I’m ready to help!  \nTo infer a causal graph and draft counterfactual explanations, I’ll need a bit more information from you:\n\n1. **List of Variables** – What are the observed variables (e.g., treatment, outcome, covariates)?\n2. **Any Known Relations** – Are there any constraints or known causal relationships you already suspect (e.g., “X causes Y”, “Z is a confounder”)?\n3. **Data or Context** – Do you have a dataset, or is this a purely theoretical scenario? If you have data, what format is it in?\n\nOnce I have those details, I can:\n\n- Build a directed acyclic graph (DAG) that captures the causal structure.\n- Identify potential confounders, mediators, and colliders.\n- Construct counterfactual explanations (e.g., “If X were set to …, what